In [1]:
import pyvisa,time
from decode_scope_data import LecroyScopeData
import numpy as np
from scipy.integrate import simpson

In [3]:
#Modify the following global variables according to the model
DSO_RSC = "TCPIP0::10.195.48.42::inst0::INSTR"

rm = pyvisa.ResourceManager('@py')
dso = rm.open_resource(DSO_RSC)
dso.timeout = 30000 #default value is 2000(2s)
dso.chunk_size = 20*1024*1024 #default value is 20*1024(20k bytes)

print(dso.query("*IDN?").strip())

Lecory,T3DSO2502A,T0101C21490989,5.0.1.3.9R6


SiPMT initialization

In [ ]:
#set the sequence
dso.write("ACQuire:SEQuence ON")
dso.write("ACQuire:SEQuence:COUNt 500")

dso.write("TRIG:MODE SINGLE")
while dso.query("TRIGGER:STAT?").strip() != 'Stop':
    pass

dso.write("HISTORY ON")
dso.write("HISTORY:PLAY FORWards")

len_history = int(dso.query(str("HISTORy:FRAMe?")).strip())
print(len_history)

28

In [ ]:
if dso.query("HISTORy?").strip() == 'ON':
    with open("./txt_data/SiPMT_data_scMirror.txt","w", encoding="utf-8") as f:
        for i in range(1,len_history):
            dso.write(f"HISTORy:FRAMe {i}")

            dso.write("WAV:SOUR C1")
            dso.write("WAV:PREamble?")
            recv_preamble = dso.read_raw()

            dso.write("WAV:DATA?")
            recv_data = dso.read_raw()

            waveform = LecroyScopeData(recv_preamble,recv_data)
            voltage = waveform.y
            time = waveform.x

            m = max(voltage)
            index_max = np.where(m == voltage)[0][0]
            ground = np.average(voltage[0:index_max-10])
            y = voltage-ground
            integral = simpson(y,time)

            out = str(m) + ',' + str(integral)
            f.write(out + '\n')
else:
    print('History: OFF')